In [8]:
# Import Data 
import datetime
import csv
import copy
import dateutil.parser
import sys
sys.path.insert(1,'../')
import Tools.data_processing as dp
# In the future we should use multiple files for annotating as some anomaly types conflict: 
# Single_point/Plummeting/Skyrocketing/TurbSpike in one file 
# Titled/3Peak/ NormalPeak in another file 
# Peak (reguardless of anomlous status) 
# spikes_fname = '../Data/data_annotating/non_processed_annotated/_.csv'
# anomalous_peaks_fname = '../Data/data_annotating/non_processed_annotated/_.csv'
# normal_peaks_fname = '../Data/data_annotating/non_processed_annotated/_.csv'

# fname1 = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/annotated_data/turb_pp/turb_PP_Labeled_0k-100k-2.csv'
# fname2 = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/annotated_data/turb_pp/turb_PP_Labeled_100k-200k-2.csv'
# fname3 = '/Users/zachfogg/Desktop/DB-SRRW/Data/manual_annotating_data/annotated_data/turb_pp/turb_PP_Labeled_200k-300k-2.csv'
# fname1 = "/Users/zachfogg/Desktop/DB-SRRW/Data/labeled_data/messy_labeled_data/fDOM/fDOM_Plummeting_Peaks/fDOM_plum_0k-100k-labeled.csv"
# fname2 = "/Users/zachfogg/Desktop/DB-SRRW/Data/labeled_data/messy_labeled_data/fDOM/fDOM_Plummeting_Peaks/fDOM_plum_100k-200k-labeled.csv"
# fname3 = "/Users/zachfogg/Desktop/DB-SRRW/Data/labeled_data/messy_labeled_data/fDOM/fDOM_Plummeting_Peaks/fDOM_plum_200k-300k-labeled.csv"
fname1 = "/Users/zachfogg/Desktop/DB-SRRW/Data/labeled_data/messy_labeled_data/fDOM/fDOM_Phantom_Peaks/fDOM_PP_smoothed_0k-100k-labeled.csv"
fname2 = "/Users/zachfogg/Desktop/DB-SRRW/Data/labeled_data/messy_labeled_data/fDOM/fDOM_Phantom_Peaks/fDOM_PP_smoothed_100k-200k-labeled.csv"
fname3 = "/Users/zachfogg/Desktop/DB-SRRW/Data/labeled_data/messy_labeled_data/fDOM/fDOM_Phantom_Peaks/fDOM_PP_smoothed_200k-300k-labeled.csv"
# fname1 = '/Users/zachfogg/Desktop/newly_lab_turb_PPP/turb_PPP_0k-100K.csv'
# fname2 = '/Users/zachfogg/Desktop/newly_lab_turb_PPP/turb_PPP_100k-200K.csv'
# fname3 = '/Users/zachfogg/Desktop/newly_lab_turb_PPP/turb_PPP_200k-300K.csv'
# out_file_fname = './data_annotating/processed_annotated/timeseries_0-100000k-annoated.csv'

In [9]:
# Separate data in fDOM, stage, turb 
turb_data = []
stage_data= []
fDOM_data = []
with open(fname1, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    next(csv_reader,None) # skip headers 
    count = 0
    for row in csv_reader: 
        if count%3 == 0:
            fDOM_data.append([row[1],row[2],row[3]])
        elif count%3 == 1:
            stage_data.append([row[1],row[2],row[3]])
        elif count%3 == 2:
            turb_data.append([row[1],row[2],row[3]])
        count +=1 
    csv_file.close()
with open(fname2, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    next(csv_reader,None) # skip headers 
    count = 0
    for row in csv_reader: 
        if count%3 == 0:
            fDOM_data.append([row[1],row[2],row[3]])
        elif count%3 == 1:
            stage_data.append([row[1],row[2],row[3]])
        elif count%3 == 2:
            turb_data.append([row[1],row[2],row[3]])
        count +=1 
    csv_file.close()
with open(fname3, newline='') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter = ',')
    next(csv_reader,None) # skip headers 
    count = 0
    for row in csv_reader: 
        if count%3 == 0:
            fDOM_data.append([row[1],row[2],row[3]])
        elif count%3 == 1:
            stage_data.append([row[1],row[2],row[3]])
        elif count%3 == 2:
            turb_data.append([row[1],row[2],row[3]])
        count +=1 
    csv_file.close()

In [10]:
# Check data read
print(len(fDOM_data))
print(len(turb_data))
print(len(stage_data))

229620
229620
229620


In [11]:
# Determine lable names 
discovered = []
for row in fDOM_data:
    val = row[2]
    if val not in discovered:
        discovered.append(val)
print(discovered)   

['', 'left_base', 'NPP', 'right_base', 'PP']


In [88]:
# Merge turb PPP and PP data
print(len(turb_data), len(turb_data_PPP))
for i,row in enumerate(turb_data_PPP):
    if row[2] == 'PPP':
        if turb_data[i][2] == 'nt_pp' or turb_data[i][2] == 'sky_dr_st' or turb_data[i][2] == 'PPP':
            turb_data[i][2] = 'PPP'
        else:
            print(turb_data[i],'   ', turb_data_PPP[i])
            break
    elif row[2] == 'NPP' and turb_data[i][2] != 'nt_pp' and turb_data[i][2] != 'sky_dr_st':
        print(turb_data[i],'   ', turb_data_PPP[i])
    elif row[2] == 'PP' and turb_data[i][2] != 't_pp' :
        print(turb_data[i],'   ', turb_data_PPP[i])


229620 229620


In [12]:
turb_data = [[row[0],row[1],row[2],i] for i,row in enumerate(turb_data)]
fDOM_data = [[row[0],row[1],row[2],i] for i,row in enumerate(fDOM_data)]
# Filter for entries that have a label 
fDOM_data_filtered = list(filter(lambda row: row[2] != '', fDOM_data))
turb_data_filtered = list(filter(lambda row: row[2] != '', turb_data))

# Convert ISO timestamp format to python datetime obj and then string
fDOM_data_converted = [[dateutil.parser.isoparse(row[0][:-5]),row[1],row[2],row[3]] for row in fDOM_data_filtered]
turb_data_converted = [[dateutil.parser.isoparse(row[0][:-5]),row[1],row[2],row[3]] for row in turb_data_filtered]

In [13]:
num_each = {}
for row in fDOM_data_filtered:
    if row[2] in num_each:
        num_each[row[2]] = num_each[row[2]]+ 1
    else:
        num_each[row[2]] = 1
for key in num_each:
    print(key, " : ", num_each[key])
    


left_base  :  408
NPP  :  441
right_base  :  409
PP  :  73


In [14]:
# Map to convert label names
# map_labels = {'intf' : 'NPLP', 'nplum_nintf' : 'NPLP', 'plum' : 'PLP'}
map_labels = {'NPP' : 'NPP', 'PP' : 'PP'}



relevant_points = list(map_labels.keys())
out_f = '../Data/labeled_data/ground_truths/fDOM/fDOM_Phantom_Peaks/julian_time/fDOM_PP_0k-300k.csv'

fDOM_data_final = list(filter(lambda row: row[2] in relevant_points, fDOM_data_converted))

with open(out_f, 'w', newline = '') as f: 
    writer = csv.writer(f, delimiter = ',')
    writer.writerow(['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak'])
    for row in fDOM_data_final:
        writer.writerow([dp.datetime_to_julian(row[0]), row[1], map_labels[row[2]], row[3]])
    f.close()
    
# Write out turb pp lables: [timestamp, indx, label]
out_f = '../Data/labeled_data/ground_truths/fDOM/fDOM_Phantom_Peaks/datetime/fDOM_PP_0k-300k.csv'
# Filter to just pp and npp 
with open(out_f, 'w', newline = '') as f: 
    writer = csv.writer(f, delimiter = ',')
    writer.writerow(['timestamp_of_peak', 'value_of_peak','label_of_peak','idx_of_peak'])
    for row in fDOM_data_final:
        writer.writerow([row[0],row[1],map_labels[row[2]],row[3]])
    f.close()